In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  # Seaborn enhances the aesthetics of matplotlib plots
import pandas as pd
import xarray as xr
import random 

from tempest import casestudy
from tempest import grid
from tempest import storm_tracker
from tempest import handler
from tempest import joint_distrib

from tempest.plots.hist import simple_hist
settings_path = 'settings/sam3d.yaml'

In [2]:
# Instantiate CaseStudy by passing the settings. 
# Should also create appropriate directories
hdlr = handler.Handler(settings_path)
cs = casestudy.CaseStudy(hdlr, overwrite = False ,verbose = False)
st = storm_tracker.StormTracker(cs, overwrite_storms = False, overwrite = False, verbose = True) #overwrite = True is super long, computes growth rate (triangle fit)

Data loaded from /homedata/mcarenso/shear/SAM3d_Tropics/var_id_days_i_t.json
Loading storms...
loading storms from netcdf
Time elapsed for loading storms: 1.33 seconds


In [3]:
duration_min = 10
surfmaxkm2_min = 25000
save_storms_path = st.settings["DIR_DATA_OUT"]+f"save_storms_dmin{duration_min}_smin{surfmaxkm2_min}.nc"
if False : 
    storms = xr.open_dataset(st.file_storms)
    storms = storms.where(storms.INT_duration > 10, drop = True) # 1min
    storms = storms.where(storms.INT_surfmaxkm2_241K> 25000, drop = True)
    storms.to_netcdf(save_storms_path)
    storms.close
else : 
    storms = xr.open_dataset(save_storms_path)

In [4]:
storms

<xarray.Dataset>
Dimensions:              (DCS_number: 6737, time: 911)
Coordinates:
  * DCS_number           (DCS_number) int64 153377 153378 ... 297410 297848
  * time                 (time) int64 817681 817682 817683 ... 818590 818591
Data variables: (12/53)
    INT_TbMin            (DCS_number) float64 ...
    INT_UTC_timeEnd      (DCS_number) float64 ...
    INT_UTC_timeInit     (DCS_number) float64 ...
    INT_classif          (DCS_number) float64 ...
    INT_classif_JIRAK    (DCS_number) float64 ...
    INT_classif_MADDOX   (DCS_number) float64 ...
    ...                   ...
    LC_tbavg_241K        (DCS_number, time) float64 ...
    LC_tbmin             (DCS_number, time) float64 ...
    LC_velocity          (DCS_number, time) float64 ...
    LC_x                 (DCS_number, time) float64 ...
    LC_y                 (DCS_number, time) float64 ...
    QCgeo_IRimage        (DCS_number, time) float64 ...

In [5]:
## FileTracking is ft
ft = storms[[ 
    "INT_UTC_timeInit", "INT_UTC_timeEnd", "INT_duration", "INT_surfcumkm2_241K", "INT_velocityAvg", ## General characteristics
    "LC_lon", "LC_lat", "LC_UTC_time", "LC_ecc_241K", "LC_orientation_241K", "LC_surfkm2_241K", "LC_tb_90th", "LC_velocity" ## General characteristics
             ]]

In [6]:
test = xr.open_dataset("/bdd/DYAMOND/SAM-4km/OUT_3D/DYAMOND_9216x4608x74_7.5s_4km_4608_0000001440_PP.nc")
targets = [300, 2000, 4000, 8000, 12000]
z_indices = [np.abs(test.z.values - target).argmin() for target in targets]
print(z_indices)
test.close()

[6, 22, 29, 37, 45]


## 3d mode


In [78]:
## Valid times for 3d var are %6==0 
ft_3d = ft.sel(time=ft.time[(ft.time - st.i_t_start + cs.settings["TIME_RANGE"][0]) % 6 == 0])
iDCS = random.choice(ft_3d.DCS_number.values)

def get_frame_data(st, ft, iDCS):
    """
    returns everything to compute the frame and fields of a DCS over it's lifetime
    start and end corresponds to filetracking time index
    time_array corresponds to i_t of global variables (seg_mask, precip, U,...)
    """

    storm = ft.sel(DCS_number = iDCS)
    start = np.max([0,int(storm.INT_UTC_timeInit.values/st.settings["NATIVE_TIMESTEP"])-st.i_t_start])
    end = 1+int(storm.INT_UTC_timeEnd.values/st.settings["NATIVE_TIMESTEP"])-st.i_t_start
    lon_array, lat_array = storm.LC_lon[start:end].values, storm.LC_lat[start:end].values
    speed_array, time_array = storm.LC_velocity[start:end].values, storm.LC_UTC_time[start:end].values/1800 - st.i_t_start

    assert time_array[0]==start
    assert time_array[-1] +1 ==end

    time_array = time_array +st.settings["TIME_RANGE"][0]
    return start, end, lon_array, lat_array, speed_array, time_array


def get_valid_3d(cs):
    valid_3d = []
    for key in cs.days_i_t_per_var_id["U"].keys():
        for vt in cs.days_i_t_per_var_id["U"][key]:
            valid_3d.append(vt)
    return valid_3d

valid_3d = get_valid_3d(cs)


def get_frame_data_3d(st, ft, iDCS):
    """
    returns everything to compute the frame and fields of a DCS over it's lifetime
    start and end corresponds to filetracking time index
    time_array corresponds to i_t of global variables (seg_mask, precip, U,...)
    """

    storm = ft.sel(DCS_number = iDCS)
    start = np.max([0,int(storm.INT_UTC_timeInit.values/st.settings["NATIVE_TIMESTEP"])-st.i_t_start])
    end = 1+int(storm.INT_UTC_timeEnd.values/st.settings["NATIVE_TIMESTEP"])-st.i_t_start
    lon_array, lat_array = storm.LC_lon[start:end].values, storm.LC_lat[start:end].values
    speed_array, time_array = storm.LC_velocity[start:end].values, storm.LC_UTC_time[start:end].values/1800 - st.i_t_start

    # Initialize speed arrays with zeros
    speed_lon = np.zeros_like(lon_array)
    speed_lat = np.zeros_like(lat_array)
    for i in range(1, len(lon_array) - 1):
        speed_lon[i] = (lon_array[i + 1] - lon_array[i - 1]) / 2 #*delta_t
        speed_lat[i] = (lat_array[i + 1] - lat_array[i - 1]) / 2

    assert time_array[0]==start
    assert time_array[-1] +1 ==end

    time_array = time_array +st.settings["TIME_RANGE"][0]
    mask = np.isin(time_array, list(valid_3d))

    return start, end, lon_array[mask], lat_array[mask], speed_array[mask], time_array[mask], speed_lon[mask], speed_lat[mask]


def get_extent_slices(lons, lats, large_scale_frame_size=4):
    lon_min, lon_max = np.min(lons)-large_scale_frame_size, np.max(lons)+large_scale_frame_size
    lat_min, lat_max = np.min(lats)-large_scale_frame_size, np.max(lats)+large_scale_frame_size
    extent = [lon_min, lon_max, lat_min, lat_max]
    slice_lon = slice(lon_min, lon_max)
    slice_lat = slice(lat_min, lat_max)
    return extent, slice_lon, slice_lat


start, end, lons, lats, speeds, times = get_frame_data(st, ft, iDCS)
start, end, lons_3d, lats_3d, speeds_3d, times_3d, speed_lon_3d, speed_lat_3d = get_frame_data_3d(st, ft, iDCS)
extent, slice_lon, slice_lat = get_extent_slices(lons_3d, lats_3d)

print(times_3d)

[1074. 1080. 1086. 1092.]


In [95]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.feature as cfeature

t = len(times_3d)

# Initialize figure and axis
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection': ccrs.PlateCarree()})

colors = [(1, 1, 1, 0), (0.8, 0.8, 0.8, 0.3), (0.5, 0.5, 0.5, 1)]  # RGBA for light grey to very light grey (with transparency)
cmap = LinearSegmentedColormap.from_list("light_grey_scale", colors)
prec_cmap =  plt.get_cmap("nipy_spectral")
w_cmap = plt.get_cmap("viridis")
prec_cmap.set_bad(color=(1, 1, 1, 0))  # I think that's useless but unsure the way maps are superimposed
cmap.set_bad(color=(1, 1, 1, 0))  # Transparent for NaNs

# Define a function to update the plot for each frame in the animation
def animate(i):
    ax.clear()  # Clear the previous frame

    le_dico_long = {"latitude" : slice_lat, "longitude" : slice_lon}
    le_dico_court = {    "lat" : slice_lat,       "lon" : slice_lon}


    # plot central cloud with contour TODO a func
    seg = hdlr.load_seg(times_3d[i])[0].sel(le_dico_long)
    nan_mask = np.isnan(seg)
    output = np.full_like(seg, 0.5, dtype = float)
    output[nan_mask] = 0
    output[seg == iDCS] = 1
    mesh = ax.contour(output, levels=[0.75], colors='red', linewidths=0.5, linestyles='solid', extent = extent, transform=ccrs.PlateCarree()) #
    output[seg == iDCS] = 0.5
    ax.imshow(output, origin = 'lower', cmap=cmap, interpolation='nearest', norm=mcolors.Normalize(vmin=0, vmax=1), extent = extent, transform=ccrs.PlateCarree())

    ## trajectory, speed vector
    ax.plot(lons_3d[:i+1], lats_3d[:i+1], marker='o', color='k', label='Trajectory Path', markersize=1)
    if i != start and i!= end-1 : 
        print(np.sqrt(speed_lon_3d[i]**2 + speed_lat_3d[i]**2))
        ax.quiver(lons_3d[i], lats_3d[i], speed_lon_3d[i], speed_lat_3d[i], scale = 0.2/np.sqrt(speed_lon_3d[i]**2 + speed_lat_3d[i]**2)) #

    ### W ### 
    # list_w = [hdlr.load_var(cs, "W", times_3d[i], z_idx = z, sel_dict = le_dico_court) for z in z_indices]
    # for w in list_w:
    #     ax.imshow(w.W[0,0].values, origin = 'lower', cmap=w_cmap, interpolation='nearest', alpha = 0.3, norm=mcolors.Normalize(vmin=-1, vmax=1), extent = extent, transform=ccrs.PlateCarree()) #
    #     break

    ### U & V Hodograph ###
    list_u = [hdlr.load_var(cs, "U", times_3d[i], z_idx = z, sel_dict = le_dico_court) for z in z_indices]
    list_v = [hdlr.load_var(cs, "V", times_3d[i], z_idx = z, sel_dict = le_dico_court) for z in z_indices]
    hodo_colors = ["blue", "orange", "green", "red", "purple"]
    for u, v, La_Couleur_De_L_hodographe  in zip(list_u, list_v, hodo_colors):
        ## So what do we plot haha it depends on a radius ? square ? cloud_mask ? 
        u_clean, v_clean = u.U[0,0].values, v.V[0,0].values
        u_mean, v_mean = np.mean(u_clean), np.mean(v_clean)
        print(np.sqrt(u_mean**2 + v_mean**2))
        ax.quiver(lons_3d[i], lats_3d[i], u_mean, v_mean, color=La_Couleur_De_L_hodographe, scale = 500/np.sqrt(u_mean**2 + v_mean**2)) #, angles='xy', scale_units='xy'


    # plot prec (threshold, percentile based?)
    prec = hdlr.load_var(cs, "Prec", times_3d[i]).sel(le_dico_court).values
    prec_masked = np.ma.masked_less_equal(prec, 3)
    ax.imshow(prec_masked, origin = 'lower', cmap = prec_cmap, interpolation='nearest', alpha = 0.3, extent = extent, transform=ccrs.PlateCarree())


    # this grid could be obtained more robustly maybe with seg directly..
    ## ax things
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.set_xticks(np.linspace(slice_lon.start, slice_lon.stop, num=5), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(slice_lat.start, slice_lat.stop, num=5), crs=ccrs.PlateCarree())

    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.coastlines(resolution='10m')
    ax.set_aspect('auto')
    ax.set_title(f'{hdlr.i_t_to_nice_datetime(times_3d[i])}', loc='center')
    return mesh,


plt.close('all')

# Create the animation
ani = animation.FuncAnimation(fig, animate, frames=t, blit=True)

# Display the animation in the Jupyter notebook
from IPython.display import HTML
HTML(ani.to_jshtml())

0.13647344063956557
9.494559327604465
9.934355155158181
6.788310952703426
4.112092114376912
8.562792011222067
0.13647344063956557
9.494559327604465
9.934355155158181
6.788310952703426
4.112092114376912
8.562792011222067
0.13647344063956557
9.494559327604465
9.934355155158181
6.788310952703426
4.112092114376912
8.562792011222067
0.13647344063956557
9.494559327604465
9.934355155158181
6.788310952703426
4.112092114376912
8.562792011222067
0.10920164833921142
8.72281819912243
9.056321708295489
5.959856882860822
4.104203381907185
8.475221893274206
0.05024937810561584
7.599521175262312
7.780961613257724
5.166282381128114
3.669001158105281
8.40597215907889
0.018027756377320302
6.507935838277727
6.455395744193467
4.334713337952372
3.3278672030059315
8.392849282608976


In [75]:
slice_lon, slice_lat

(slice(303.72, 313.16, None), slice(4.119999999999999, 13.03, None))

In [ ]:
# ani.save(f'iDCS_{iDCS}.mp4', writer='ffmpeg', fps=5)

### Maybe old

In [21]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.feature as cfeature

t = len(times)

# Initialize figure and axis
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': ccrs.PlateCarree()})

colors = [(1, 1, 1, 0), (0.8, 0.8, 0.8, 0.3), (0.5, 0.5, 0.5, 1)]  # RGBA for light grey to very light grey (with transparency)
cmap = LinearSegmentedColormap.from_list("light_grey_scale", colors)
prec_cmap =  plt.get_cmap("nipy_spectral")

prec_cmap.set_bad(color=(1, 1, 1, 0))  # I think that's useless but unsure the way maps are superimposed
cmap.set_bad(color=(1, 1, 1, 0))  # Transparent for NaNs


# Define a function to update the plot for each frame in the animation
def animate(i):
    ax.clear()  # Clear the previous frame

    le_dico_long = {"latitude" : slice_lat, "longitude" : slice_lon}
    le_dico_court = {    "lat" : slice_lat,       "lon" : slice_lon}

    ## plot prec (threshold, percentile based?)
    prec = hdlr.load_var(cs, "Prec", times[i]).sel(le_dico_court).values
    prec_masked = np.ma.masked_less_equal(prec, 3)
    ax.imshow(prec_masked, origin = 'lower', cmap = prec_cmap, interpolation='nearest', alpha = 0.5, extent = extent, transform=ccrs.PlateCarree())

    ## plot central cloud with contour TODO a func
    seg = hdlr.load_seg(times[i])[0].sel(le_dico_long)
    nan_mask = np.isnan(seg)
    output = np.full_like(seg, 0.5, dtype = float)
    output[nan_mask] = 0
    output[seg == iDCS] = 1
    ax.contour(output, levels=[0.75], colors='red', linewidths=0.5, linestyles='solid', extent = extent, transform=ccrs.PlateCarree())
    output[seg == iDCS] = 0.5
    mesh = ax.imshow(output, origin = 'lower', cmap=cmap, interpolation='nearest', norm=mcolors.Normalize(vmin=0, vmax=1), extent = extent, transform=ccrs.PlateCarree())

    ## trajectory TODO speed vector
    ax.plot(lons[:i], lats[:i], marker='o', color='k', label='Trajectory Path', markersize=1)
    if i != 0 and i!= t-1 : 
        ax.quiver(lons[i], lats[i], (lons[i+1]-lons[i-1])/2, (lats[i+1] - lats[i-1])/2)
        
    ## plot winds : actually its pretty hard and not relly informative.. better work on clustering precip features and cold pool. 

    # this grid could be obtained more robustly maybe with seg directly..
    ## ax things
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.set_xticks(np.linspace(slice_lon.start, slice_lon.stop, num=5), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(slice_lat.start, slice_lat.stop, num=5), crs=ccrs.PlateCarree())

    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.coastlines(resolution='10m')

    ax.set_title(f'{hdlr.i_t_to_nice_datetime(times[i])}', loc='center')
    return mesh,


plt.close('all')

# Create the animation
ani = animation.FuncAnimation(fig, animate, frames=t, blit=True)

# Display the animation in the Jupyter notebook
from IPython.display import HTML
HTML(ani.to_jshtml())

/home/mcarenso/virtual_envs/PyHD/lib/python3.10/site-packages/matplotlib/quiver.py:649: RuntimeWarning: divide by zero encountered in double_scalars
  length = a * (widthu_per_lenu / (self.scale * self.width))
/home/mcarenso/virtual_envs/PyHD/lib/python3.10/site-packages/matplotlib/quiver.py:649: RuntimeWarning: invalid value encountered in multiply
  length = a * (widthu_per_lenu / (self.scale * self.width))


## Get time, lat, lon 

In [6]:
import matplotlib.colors as mcolors
import cartopy.crs as ccrs

iDCS = random.choice(ft_3d.DCS_number.values)

def get_frame_data(st, ft, iDCS):
    """
    returns everything to compute the frame and fields of a DCS over it's lifetime
    start and end corresponds to filetracking time index
    time_array corresponds to i_t of global variables (seg_mask, precip, U,...)
    """

    storm = ft.sel(DCS_number = iDCS)
    start = np.max([0,int(storm.INT_UTC_timeInit.values/st.settings["NATIVE_TIMESTEP"])-st.i_t_start])
    end = 1+int(storm.INT_UTC_timeEnd.values/st.settings["NATIVE_TIMESTEP"])-st.i_t_start
    lon_array, lat_array = storm.LC_lon[start:end].values, storm.LC_lat[start:end].values
    speed_array, time_array = storm.LC_velocity[start:end].values, storm.LC_UTC_time[start:end].values/1800 - st.i_t_start

    assert time_array[0]==start
    assert time_array[-1] +1 ==end

    time_array = time_array +st.settings["TIME_RANGE"][0]
    return start, end, lon_array, lat_array, speed_array, time_array

start, end, lons, lats, speeds, times = get_frame_data(st, ft, iDCS)

def get_extent_slices(lons, lats, large_scale_frame_size=4):
    lon_min, lon_max = np.min(lons)-large_scale_frame_size, np.max(lons)+large_scale_frame_size
    lat_min, lat_max = np.min(lats)-large_scale_frame_size, np.max(lats)+large_scale_frame_size
    extent = [lon_min, lon_max, lat_min, lat_max]
    slice_lon = slice(lon_min, lon_max)
    slice_lat = slice(lat_min, lat_max)
    return extent, slice_lon, slice_lat

extent, slice_lon, slice_lat = get_extent_slices(lons, lats)

print(times)

def load_plot_cloud(i_t, dict_frame, ax, cmap):
    seg = hdlr.load_seg(i_t)[0].sel(dict_frame)
    nan_mask = np.isnan(seg)
    output = np.full_like(seg, 0.5, dtype = float)
    output[nan_mask] = 0
    output[seg == iDCS] = 1
    ax.contour(output, levels=[0.75], colors='red', linewidths=0.5, linestyles='solid', extent = extent, transform=ccrs.PlateCarree())
    output[seg == iDCS] = 0.5
    mesh = ax.imshow(output, origin = 'lower', cmap=cmap, interpolation='nearest', norm=mcolors.Normalize(vmin=0, vmax=1), extent = extent, transform=ccrs.PlateCarree())
    return mesh
    

[1166. 1167. 1168. 1169. 1170. 1171. 1172. 1173. 1174. 1175. 1176. 1177.
 1178. 1179. 1180. 1181. 1182. 1183. 1184. 1185. 1186. 1187. 1188. 1189.]


## 3d winds avail


In [ ]:
import matplotlib.colors as mcolors
import cartopy.crs as ccrs





def load_plot_cloud(i_t, dict_frame, ax, cmap):
    seg = hdlr.load_seg(i_t)[0].sel(dict_frame)
    nan_mask = np.isnan(seg)
    output = np.full_like(seg, 0.5, dtype = float)
    output[nan_mask] = 0
    output[seg == iDCS] = 1
    ax.contour(output, levels=[0.75], colors='red', linewidths=0.5, linestyles='solid', extent = extent, transform=ccrs.PlateCarree())
    output[seg == iDCS] = 0.5
    mesh = ax.imshow(output, origin = 'lower', cmap=cmap, interpolation='nearest', norm=mcolors.Normalize(vmin=0, vmax=1), extent = extent, transform=ccrs.PlateCarree())
    return mesh
    
    start, end, lons, lats, speeds, times = get_frame_data(st, ft, iDCS)
extent, slice_lon, slice_lat = get_extent_slices(lons, lats)

print(times)